In [48]:
%pip install -qU langchain-openai langchain-community langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os

from dotenv import load_dotenv
load_dotenv("../.azure.env")

True

In [34]:
import base64

def get_screen_text(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    screens = [line.split("'")[1] for line in lines]   
    return screens

def load_image(image_path: str) -> dict:    
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
    
    image_base64 = encode_image(image_path)
    return image_base64

In [46]:
system_prompt="""
You are a teaching assistant that invigilate a computer practical test.
Students do not allow to
1. Use any AI tools during the test.
2. No communication with massager such as WhatApps, Telegram, ...
3. Highlight any suspicious acts.


You are tasked with assisting students during a computer-based practical test as an invigilator. Your responsibilities include ensuring rules are adhered to, monitoring the test environment, and identifying any suspicious behavior. Follow these guidelines.

- **Prohibited Actions:**
  1. Students are **not allowed to ask for answer from any AI tools** during the test.
  2. Students must **not communicate using messaging platforms** such as WhatsApp, Telegram, or similar.

- **Monitoring and Reporting:**
  - Actively observe and monitor students during the test for any potential rule violations.
  - **Identify and document suspicious acts** (e.g., switching between unpermitted applications, unusual keyboard/mouse activity, or actions suggesting external communication).
  - Remain impartial—do not accuse students without proper evidence. Simply document and report to the test supervisor for review.

# Steps

1. Look for specific indicators of rule violations:
   - Switching to unauthorized software or pages unrelated to the test.
   - Use of chat or messaging platforms.
   - Unusual or repetitive screen or application switching patterns.
2. Document any suspicious behavior:
   - Note the details of the incident: student identity, time, and nature of the act.
   - Avoid direct confrontation unless an immediate correction (e.g., preventing further cheating) is necessary.

# Output Format

Provide a report in the following structure when highlighting suspicious acts:

- **Details of Act:** [Describe the suspicious act, e.g., "Student attempted to open WhatsApp during the test."]
- **Evidence:** [Attach or describe any observed evidence, such as screenshots or detailed observations.]

# Example

- **Details of Act:** Student switched between the test application and a browser with AI-related tools multiple times. Continued to move between tabs despite being reminded of the rules.  
- **Evidence:** Observed screen activity showing AI tool logos in the browser.

# Notes

- Do not assume guilt—report only what you directly observe with objective details.


"""

validate_screen_system_prompt = """
Evaluate and review invigilator comments from a computer-based practical test to ensure compliance with test guidelines and identify indications of policy violations or suspicious activities.

Students are **allowed to browse the internet using web browsers** but are **not allowed** to use AI tools except for **"Azure Open AI ChatGPT"** hosted on the domain `*.azurestaticapps.net`. Messaging tools (e.g., WhatsApp, Facebook Messenger, etc.) or any form of communication are also strictly prohibited during the test.

# Steps

1. **Guideline Compliance**: 
   - Identify if the activity referenced in the comment is permitted:
     - Internet browsing is allowed, but unauthorized AI tools are prohibited.
     - Only "Azure Open AI ChatGPT" on `*.azurestaticapps.net` domains is authorized for AI-based help.
     - Messaging platforms (e.g., WhatsApp, Facebook Messenger, etc.) or other communication tools are prohibited.
   - Verify that no activities violate the prescribed rules.
2. **Flag Suspicious Activities**:
   - Highlight comments suggesting unauthorized AI tools or messaging platforms were used.
   - Look out for hints of collaboration, external assistance, or circumvention of the rules.
   - Monitor whether browsing activities align with expected test-related tasks or objectives.
3. **Generate Feedback**:
   - If a rule is violated:
     - Indicate the violated guideline and provide reasoning.
   - If a situation is unclear:
     - Recommend further investigation with a rationale.
   - If compliant:
     - Confirm there are no violations, and clarify why the action was considered acceptable.

# Output Format

Return an organized evaluation for each invigilator comment in the following JSON format:

```json
{
  "reviewed_comment": "<comment_text>",
  "status": "<compliant|non-compliant>",
  "issue_detected": "<description_of_violation_or_suspicious_activity>",
  "suggestions": "<optional_step_for_further_investigation_or_action>"
}
```

- **`reviewed_comment`**: The text of the comment provided by the invigilator.
- **`status`**: Whether the comment indicates compliance (`compliant`) or a breach of rules (`non-compliant`).
- **`issue_detected`**: A description of the suspicious activity if the comment is `non-compliant`, or "None" if the comment is `compliant`.
- **`suggestions`**: Optional recommendations for further investigation or required actions.

# Examples

### Example 1:
Input Comment:  
"Student accessed https://chat.openai.com and used the site frequently during the test."

Output:
```json
{
  "reviewed_comment": "Student accessed https://chat.openai.com and used the site frequently during the test.",
  "status": "non-compliant",
  "issue_detected": "Use of unauthorized AI tool (ChatGPT on openai.com).",
  "suggestions": "Verify browsing logs and restrict access to unauthorized sites in future tests."
}
```

### Example 2:  
Input Comment:  
"Student opened a page on documentation.microsoft.com and referred to it for task-related information."

Output:  
```json
{
  "reviewed_comment": "Student opened a page on documentation.microsoft.com and referred to it for task-related information.",
  "status": "compliant",
  "issue_detected": "None",
  "suggestions": "None"
}
```

### Example 3:  
Input Comment:  
"Student appeared to open WhatsApp Web in their browser during the test."

Output:  
```json
{
  "reviewed_comment": "Student appeared to open WhatsApp Web in their browser during the test.",
  "status": "non-compliant",
  "issue_detected": "Access to a prohibited messaging platform (WhatsApp Web).",
  "suggestions": "Investigate usage logs and warn the student about policy compliance."
}
```

# Notes

- Be vigilant about edge cases, such as scenarios where external links were opened unintentionally or multitasking was misinterpreted.  
- Ensure consistent interpretation across varying activities to maintain fairness to the students.  
- Apply objectivity and reasonableness to distinguish between intentional violations and permissible actions (e.g., general web browsing for study purposes versus explicit use of restricted platforms). """

In [47]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_GPT4O_DEPLOYMENT_NAME"),
    temperature=0,
)


model: AzureChatOpenAI = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_GPT4O_DEPLOYMENT_NAME"),
    temperature=0,
)


def batch_check_screens(screens: list) -> list:
    messages = []
    print(f"Total screens: {len(screens)}")
    for i in range(0, len(screens), 50):
        batch_screens = screens[i:i+50]
        msg = model.invoke(
            [
                SystemMessage(content=system_prompt),
                HumanMessage(
                    content=[
                        {"type": "text", "text": "Please review the following images and tell me anything any irregulating."},
                        *[
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{load_image(screen)}"
                                },
                            }
                            for screen in batch_screens
                        ],
                    ]
                )
            ]
        )
        messages.append(msg.content)
    return messages


def validate_screen_comments(comment) -> str:
    msg = model.invoke(
        [
            SystemMessage(content=validate_screen_system_prompt),
            HumanMessage(
                content=[
                    {"type": "text", "text": "Please review the comments and tell me anything any irregulating."},
                    {"type": "text", "text": comment},                    
                ]
            )
        ]
    )

    return msg.content

In [44]:
from time import sleep
from tqdm import tqdm
import glob

screen_files = glob.glob("../output/*.txt")
screen_files = [
    file for file in screen_files if not file.endswith("_result.txt")]

for file_path in tqdm(screen_files, desc="Processing files"):
    raw_result_file = file_path.replace(".txt", "_raw_result.txt")
    if os.path.exists(raw_result_file):
        continue
    print(f"Processing: {file_path} and number of screens: {
          len(screen_files)}")

    screens = get_screen_text(file_path)
    screens = screens[::25]
    messages = batch_check_screens(screens)
    with open(raw_result_file, 'w') as file:
        for message in messages:
            file.write(message + "\n")
    sleep(30)

Processing files:   0%|          | 0/1 [00:00<?, ?it/s]

Processing: ../output/240410007-from 2024-11-18-05-41 to 2024-11-18-06-01.txt and number of screens: 1
Total screens: 10


Processing files: 100%|██████████| 1/1 [00:45<00:00, 45.36s/it]


In [48]:
raw_result_files = glob.glob("../output/*_raw_result.txt")
for file_path in tqdm(raw_result_files, desc="Processing raw files"):
    with open(file_path, 'r') as file:
        comment = file.read()
    result = validate_screen_comments(comment)
    print(result)
    with open(file_path.replace("_raw_result.txt", "_result.txt"), 'w') as file:
        file.write(result)
    

Processing raw files:   7%|▋         | 1/14 [00:58<12:39, 58.40s/it]

```json
[
  {
    "reviewed_comment": "Student attempted to open a new tab and search for 'Azure Open AI ChatGPT Virtual' during the test.",
    "status": "compliant",
    "issue_detected": "None",
    "suggestions": "None"
  },
  {
    "reviewed_comment": "Student navigated to the VTC website and logged into the VTC MyPortal during the test.",
    "status": "compliant",
    "issue_detected": "None",
    "suggestions": "None"
  },
  {
    "reviewed_comment": "Student attempted to open a browser tab and navigate to a search engine during the test.",
    "status": "compliant",
    "issue_detected": "None",
    "suggestions": "None"
  },
  {
    "reviewed_comment": "Based on the provided images, I did not observe any irregularities or suspicious activities. The student appears to be working within the Cisco Packet Tracer environment, focusing on the task at hand, which involves identifying physical characteristics of internetworking devices and selecting correct modules for connectivity."

Processing raw files:  14%|█▍        | 2/14 [01:58<11:55, 59.61s/it]

Based on the provided comments, here is the organized evaluation for each invigilator comment:

### Comment 1:
"Student has a browser tab open with 'Azure Open AI ChatGPT' and is attempting to access it."

```json
{
  "reviewed_comment": "Student has a browser tab open with 'Azure Open AI ChatGPT' and is attempting to access it.",
  "status": "compliant",
  "issue_detected": "None",
  "suggestions": "None"
}
```

### Comment 2:
"Student is sharing their screen via a service (as indicated by the 'Stop sharing' button at the bottom of the screen)."

```json
{
  "reviewed_comment": "Student is sharing their screen via a service (as indicated by the 'Stop sharing' button at the bottom of the screen).",
  "status": "non-compliant",
  "issue_detected": "Screen sharing could be used to communicate with others, which is against the test rules.",
  "suggestions": "Investigate usage logs and warn the student about policy compliance."
}
```

### Comment 3:
"Student has exposed an API key in the c

Processing raw files:  21%|██▏       | 3/14 [03:05<11:29, 62.66s/it]

...

Based on the provided observations, here is the evaluation of each comment in the required JSON format:

### Comment 1:
"Student has multiple browser tabs open, including 'Azure Open AI ChatGPT' and 'thankful-moss-0a2536000.5a...'. This could be suspicious as it might indicate the use of AI tools or external resources, which are not allowed during the test."

```json
{
  "reviewed_comment": "Student has multiple browser tabs open, including 'Azure Open AI ChatGPT' and 'thankful-moss-0a2536000.5a...'. This could be suspicious as it might indicate the use of AI tools or external resources, which are not allowed during the test.",
  "status": "non-compliant",
  "issue_detected": "Potential use of unauthorized AI tools or external resources.",
  "suggestions": "Verify the domain of the 'thankful-moss-0a2536000.5a...' tab to confirm if it is an unauthorized AI tool. Remind the student of the test rules regarding AI tool usage."
}
```

### Comment 2:
"Student executed a terminal command to m

Processing raw files:  93%|█████████▎| 13/14 [07:16<00:32, 32.64s/it]

Based on the provided observations, here is the evaluation of each comment:

### First Image:
```json
{
  "reviewed_comment": "The student has multiple tabs open, including 'Azure Open AI ChatGPT Vir' and 'Tokyo Live, Skyline with BGM: E' on YouTube.",
  "status": "non-compliant",
  "issue_detected": "Use of unauthorized AI tool (Azure Open AI ChatGPT Vir) and potential distraction with YouTube.",
  "suggestions": "Verify the specific domain of the AI tool to ensure it is authorized. Remind the student about the prohibition of non-test-related browsing."
}
```

### Second Image:
```json
{
  "reviewed_comment": "The student is working on a diagram using 'app.diagrams.net'.",
  "status": "compliant",
  "issue_detected": "None",
  "suggestions": "None"
}
```

### Third Image:
```json
{
  "reviewed_comment": "The student continues to work on the diagram.",
  "status": "compliant",
  "issue_detected": "None",
  "suggestions": "None"
}
```

### Fourth Image:
```json
{
  "reviewed_comment": "

Processing raw files: 100%|██████████| 14/14 [07:22<00:00, 31.63s/it]

Based on the provided observations and irregularities noted from the images, here is the detailed review in the required JSON format:

```json
[
  {
    "reviewed_comment": "Image 1: Moodle Course Page - The page shows instructions for an EA Practical Test. The instructions include logging into various platforms and using specific tools.",
    "status": "compliant",
    "issue_detected": "None",
    "suggestions": "None"
  },
  {
    "reviewed_comment": "Image 2: GitHub Dashboard - The student is logged into GitHub, which is part of the test instructions.",
    "status": "compliant",
    "issue_detected": "None",
    "suggestions": "None"
  },
  {
    "reviewed_comment": "Image 3: Microsoft Copilot - The student is using Microsoft Copilot, which is an AI tool.",
    "status": "non-compliant",
    "issue_detected": "Use of unauthorized AI tool (Microsoft Copilot).",
    "suggestions": "Verify the extent of usage and remind the student of the AI tool restrictions."
  },
  {
    "reviewed